In [1]:
import cobra
import pandas as pd
import re
import copy

In [5]:
core=pd.read_csv('ModelList.txt', sep="\t", index_col=0, header=None)
for i in core.index:
    core.rename(index={i:re.sub('\.xml','',i)}, inplace=True)
genera=core.index

In [14]:
RESULT=pd.DataFrame(columns=genera)

In [ ]:
#Test all C sources for every model
for g in genera:
    #load model
    model=cobra.io.read_sbml_model(g+'.xml')
    model.solver='glpk'
    #Count C in formula
    C_exch={}
    for i in model.metabolites:
        if isinstance(i.formula, str):
            if re.search ('C[^a-z]', i.formula):
                for r in i.reactions:
                    if re.search ('EX_',r.id):
                        C_exch[r.id]=i.formula
    for i in list(C_exch.keys()):
        if re.search('C\d+',C_exch[i]):
            p = re.compile('C(\d+)')
            C_exch[i]=p.findall(C_exch[i])[0]
        else:
             C_exch[i]=1
    
    #constraint C uptake
    coefficients=dict()
    model2=copy.deepcopy(model)
    for r in C_exch:
        coefficients[model2.reactions.get_by_id(r).reverse_variable] = C_exch[r]
    constraint=model2.problem.Constraint(0,lb=0,ub=10)
    model2.add_cons_vars(constraint)
    model2.solver.update()
    constraint.set_linear_coefficients(coefficients=coefficients)
    ref=model2.slim_optimize()
    
    #test ability to use compounds as C sources
    Csources=[]
    for n in C_exch:
        coeffs=copy.deepcopy(coefficients)
        model2=copy.deepcopy(model)
        coefficients[model2.reactions.get_by_id(n).reverse_variable]=0
        constraint=model2.problem.Constraint(0,lb=0,ub=10)
        model2.add_cons_vars(constraint)
        model2.solver.update()
        constraint.set_linear_coefficients(coefficients=coeffs)
        if model2.slim_optimize()>ref*2:
            for m in model.reactions.get_by_id(n).metabolites:
                mname=m.name
            Csources.append(mname)
    for n in Csources:
        RESULT.loc[n,g]=1
    #update output file
    RESULT.to_csv('C_sourcesGrowth_10mm.txt', sep="\t")
    print(g)
    #break